# Poisson 2D

This notebook solves the linear problem:

$$ - \Delta u = f \text{ in } \Omega = [0,1]^2$$ 

with

$$ f(x, y) = 10 \exp^{-50 [(x - 1/2)^2 + (y - 1/2)^2)]} $$

and boundary conditions given by

$$    u(x, y) = 0 \text{ for } x = 0 \text{ or } x = 1 $$

and

$$ \frac{\partial u}{\partial n}(x, y) = sin(5 x) \text{ for } y = 0 \text{ or } y = 1. $$

The purpose is to find what the issues are while setting BCs using Hermite elements and fix them. See [Fixing the boundary conditions](#Fixing-the-boundary-conditions).

In [ ]:
from dolfin import *
import matplotlib.pyplot as pl
import numpy as np
from IPython.display import Math

%matplotlib inline

parameters["form_compiler"]["cpp_optimize"] = False
parameters["form_compiler"]["optimize"] = False

def poisson(V):
    """ Solves Poisson's eq on the unit square.
    This code is based on DOLFIN's homonymous demo by Anders Logg
    Arguments:
        V: FunctionSpace.
    """

    def boundary(x):
        return near(x[0], 0) or near(x[0], 1)

    # Need to project to work around lack of evaluate_dofs() for Hermite
    u0 = project(Constant(0.0), V)
    bc = DirichletBC(V, u0, boundary)

    u = TrialFunction(V)
    v = TestFunction(V)
    f = Expression("10*exp(-50*(pow(x[0] - 0.5, 2) + pow(x[1] - 0.5, 2)))",\
                   degree=2)
    g = Expression("sin(5*x[0])", degree=2)
    a = inner(grad(u), grad(v))*dx
    L = f*v*dx + g*v*ds

    u = Function(V)
    solve(a == L, u, bc)
    return u

## Four (not so) different solutions

We use P1, P2 and P3 Lagrange elements as well as P3 Hermite elements and compare the results.

In [ ]:
mesh = UnitSquareMesh(64, 64)
W = [None] + [FunctionSpace(mesh, "Lagrange", i) for i in range(1, 4)]
lag = [None] + [poisson(W[p]) for p in range(1, 4)]

pl.figure(figsize=(8, 8))
for i in range(1, 4):
    pl.subplot(2, 2, i)
    plot(lag[i], title = "P%d Lagrange" % i, cmap='bone')
V = FunctionSpace(mesh, "Hermite", 3)
her = poisson(V)
pl.subplot(2,2,4)
_ = plot(her, title="P3 Hermite", cmap='bone')

## Relative errors

We find a consistent relative error of around 14% measured in the $H^1$ norm and 4% in the $L^2$ norm.

In [ ]:
pl.figure(figsize=(12,4))
eqs = ''#\\begin{align}'
for i in range(1,4):
    diff = project(lag[i] - her, V)
    eqs += '\\frac{||u_{\\text{l%d}} - u_{\\text{h3}}||_{L_2}}{||u_{\\text{l%d}}||_{L_2}} \\approx %.4f,\ \ \ '\
           ' \\frac{||u_{\\text{l%d}} - u_{\\text{h3}}||_{H_1}}{||u_{\\text{l%d}}||_{H_1}} \\approx %.4f \\\~ '\
           % (i, i, norm(diff, 'L2') / norm(lag[i], 'L2'), i, i, norm(diff, 'H1') / norm(lag[i], 'H1'))
    pl.subplot(1,3,i)
    plot(diff, title="|Lag%d - Her3|" % i, cmap='bone')
eqs += ''#'\\end{align}'
Math(eqs)

A final plot with some "cross-sections" at fixed ordinates for the P3 Lagrange vs Hermite elements. We need to cap the relative error because of very small denominators.

In [ ]:
xx = np.linspace(0,1,100)
pl.figure(figsize=(8,6))
error_threshold = 0.5

for y in np.linspace(0, 1, 4):
    pl.plot(xx, [min(error_threshold, np.abs(diff(x,y)/lag[3](x,y))) for x in xx],
            label="y = %.1f" % y)
    #pl.plot(xx, [diff(x,y) for x in xx], label="D@%.1f" % y)
    #pl.plot(xx, [her([x,y]) for x in xx], label="H@%.1f" % y)
    #pl.plot(xx, [lag([x,y]) for x in xx], label="L@%.1f" % y)
pl.title("Relative error (capped at %.1f) at multiple ordinates" % error_threshold)
_ = pl.legend()

# Fixing the boundary conditions

We try to use the method developed in [boundary.ipynb](boundary.ipynb) for [Poisson1D](Poisson1D.ipynb) in higher dimensions. However, issues other than rows corresponding to boundary elements seem to enter into play. In particular, it might be that some *inner* Hermite dofs are being set to 0 by mistake. See [Interior effects](#Interior-effects).

We now simplify the problem:

$$ - \Delta u = f \text{ in } \Omega = [0,1]^2$$ 

with

$$ f(x, y) = 4 $$

and boundary conditions given by

$$    u(x, y) = 2 \text{ on } \partial \Omega. $$

In [ ]:
from dolfin import *
import matplotlib.pyplot as pl
import numpy as np
%matplotlib inline

import nbimporter
from boundary import apply_dirichlet_hermite, plot_hermite_dofs

In [ ]:
from utils import fnand, fnor, one_arg

# For now, we need dense matrices for apply_dirichlet_hermite() to work
parameters['linear_algebra_backend'] = 'Eigen'
mesh = UnitSquareMesh(10, 10)
V = FunctionSpace(mesh, "Hermite", 3)

top = lambda x: near(x[1], 1.)
right = lambda x: near(x[0], 1.)
bottom = lambda x: near(x[1], 0.)
left = lambda x: near(x[0], 0.)

# Need to project to work around lack of evaluate_dofs() for Hermite
u0 = project(Constant(2.0), V)
bc = DirichletBC(V, u0, one_arg(fnor(top, right, bottom, left)))

class td_subdomain(SubDomain):
    """ Top and bottom sides of the domain. """
    def inside(self, x, on_boundary):
        # Careful using on_boundary: it's False if the DirichletBC method is 'pointwise'
        return near(x[1], 0.) or near(x[1], 1.)

class lr_subdomain(SubDomain):
    """ Top and bottom sides of the domain. """
    def inside(self, x, on_boundary):
        # Careful using on_boundary: it's False if the DirichletBC method is 'pointwise'
        return near(x[0], 0.) or near(x[0], 1.)

# We will use this to mark boundaries for natural boundary conditions
exterior_facet_domains = FacetFunction("uint", mesh)
exterior_facet_domains.set_all(1)

natural_boundary = td_subdomain()
natural_boundary.mark(exterior_facet_domains, 0)
ds = ds(subdomain_data = exterior_facet_domains)

u = TrialFunction(V)
v = TestFunction(V)
f = project(Constant(4), V)
#g = project(Constant(0.), V)
a = inner(grad(u), grad(v))*dx
F = f*v*dx # + g*v*ds

# Compute solution
A = assemble(a)
b = assemble(F)
apply_dirichlet_hermite(A, b, bc)
her = Function(V)
H = her.vector()
solve(A, H, b)

We first make sure that the routines setting the BCs know what nodes in the boundary to look at. `plot_hermite_dofs()` draws all nodes for a boundary condition which have Hermite dofs associated to them:

In [ ]:
%matplotlib inline
#plot_hermite_dofs(bc)

The solution to the problem above looks almost correct:

In [ ]:
pl.figure(figsize=(6,6))
pl.xlim((-0.1,1.1))
pl.ylim((-0.1,1.1))
plot(her, cmap='bone')
_ = pl.title("P3 Hermite")

## Interior effects

Even thought the solution seems correct, there are some strange effects. Notice the wiggly cross-sections in the following plot: might it be that (some of?) the Hermite dofs *inside* the domain are being set to zero? There are also some strange assymetries (see e.g. the lines at $y=0.7$ and $y=0.9$)

In [ ]:
xx = np.linspace(0, 1, 100)
for y in np.linspace(0.1,0.9,5):
    pl.plot(xx, [her([x, y]) for x in xx], label="$y=%.1f$" % y)
_ = pl.legend(fancybox=True, framealpha=0.8)

The following assymetry is also weird:

In [ ]:
pl.plot(xx, [her([x, x]) for x in xx], label="$u_h(x,x)$")
_ = pl.legend(fancybox=True, framealpha=0.8)

## Solution with P2 Lagrange elements

As a sanity check, here is one solution known to be correct:

In [ ]:
W = FunctionSpace(mesh, "Lagrange", 2)
u0 = Constant(2.0)
bc = DirichletBC(W, u0, one_arg(fnor(top, right, bottom, left)))

# Define variational problem
u = TrialFunction(W)
v = TestFunction(W)
f = Constant(4.)
#g = Constant(0.)
a = inner(grad(u), grad(v))*dx
F = f*v*dx #+ g*v*ds

# Compute solution
lag = Function(W)
solve(a == F, lag, bc)

In [ ]:
pl.figure(figsize=(6,6))
pl.xlim((-0.1, 1.1))
pl.ylim((-0.1, 1.1))
plot(lag, cmap='bone')
_ = pl.title("P2 Lagrange")

The cross-sections look fine:

In [ ]:
xx = np.linspace(0, 1, 100)
for y in np.linspace(0.1,0.9,5):
    pl.plot(xx, [lag([x, y]) for x in xx], label="$y=%.1f$" % y)
_ = pl.legend(fancybox=True, framealpha=0.8)

The (normalized) distance between the Lagrange and Hermite solutions is huge: 3% in $L^2$ norm and 16% in $H^1$ norm for a 10x10 mesh. However in a 20x20 the errors go down by almost an order of magnitude to 0.3% and 3.8% respectively. Is all this simply a consequence of the convergence properties of Hermite elements?

In [ ]:
diff = project(lag - her, V)
#diff.vector().abs()
i = 2
out =  '\\frac{||u_{\\text{l%d}} - u_{\\text{h3}}||_{L_2}}{||u_{\\text{l%d}}||_{L_2}} \\approx %.4f,\ \ \ '\
       ' \\frac{||u_{\\text{l%d}} - u_{\\text{h3}}||_{H_1}}{||u_{\\text{l%d}}||_{H_1}} \\approx %.4f \\\~ '\
       % (i, i, norm(diff, 'L2') / norm(lag, 'L2'), i, i, norm(diff, 'H1') / norm(lag, 'H1'))
from IPython.display import Math
Math(out)

**Nevertheless:** The fact that the difference is clearly skewed along one direction seems to indicate some problem with the implementation

In [ ]:
_ = plot(diff, title="$u_{l%d} - u_{h3}$" % i, cmap='bone')

We can plot the difference along the axes $x=y$, $x=1-y$:

In [ ]:
C = norm(lag, norm_type='l2')
pl.plot(xx, [diff(x, x)/C for x in xx], label="$(u_{l2} - u_{h3})(x,x)$")
pl.plot(xx, [diff(x, 1-x)/C for x in xx], label="$(u_{l2} - u_{h3})(x,1-x)$")
pl.title("Difference, normalized by $||u_{l2}||_{L^2}$")
_ = pl.legend(loc='lower right')